In [44]:
optim = require 'optim'
fullset = torch.load('newdata20.dat')

In [45]:
itorch.image(fullset.data[1])
print(fullset.label[1])

111	


In [46]:
shuffle = torch.randperm(fullset.size)
shuffleset = fullset
for i=1, fullset.size do
    shuffleset.data[i] = fullset.data[shuffle[i]]
    shuffleset.label[i] = fullset.label[shuffle[i]]
    end
fullset = shuffleset

In [47]:
fullset

{
  data : DoubleTensor - size: 40134x20x20
  size : 40121
  label : DoubleTensor - size: 40134
}


In [48]:
for i=1,fullset.size do
    if(fullset.label[i]<=57) then
        fullset.label[i]=fullset.label[i]-47
    else
        fullset.label[i]=fullset.label[i]-97+11;
    end
end

In [49]:
trainset = {
    size = 32000,
    data = fullset.data[{{1,32000}}]:double(),
    label = fullset.label[{{1,32000}}]
}

In [50]:
validationset = {
    size = 2159,
    data = fullset.data[{{32001,34159}}]:double(),
    label = fullset.label[{{32001,34159}}]
}

In [51]:
itorch.image(trainset.data[1])
print(trainset.label[1])

4	


In [52]:
testset = {
    size = 2000,
    data = fullset.data[{{34160,36159}}]:double(),
    label = fullset.label[{{34160,36159}}]
}

In [53]:
--[[nn = require 'nn'
model = nn.Sequential()
model:add(nn.SpatialConvolution(1, 6, 3, 3))
--net:add(nn.ReLU())                       -- non-linearity 
model:add(nn.SpatialMaxPooling(2,2,2,2))     -- A max-pooling operation that looks at 2x2 windows and finds the max.
model:add(nn.View(6*9*9))--]]  
nn = require 'nn'
model = nn.Sequential()
model:add(nn.Reshape(400))

In [54]:
--[[model:add(nn.Linear(400, 120))             -- fully connected layer (matrix multiplication between input and weights)
model:add(nn.Tanh())                       -- non-linearity 
model:add(nn.Linear(120, 36))                   -- 10 is the number of outputs of the network (in this case, 10 digits)
model:add(nn.LogSoftMax())--]]                    -- converts the output to a log-probability. Useful for classification problems

model:add(nn.Linear(400,700))
model:add(nn.Tanh())
--model:add(nn.Linear(174,131))
--model:add(nn.Tanh())
model:add(nn.Linear(700,36))
model:add(nn.LogSoftMax())

In [55]:
--model=torch.load('oldmodel.net')
criterion = nn.ClassNLLCriterion()

In [56]:
sgd_params = {
   learningRate = 1e-2,
   learningRateDecay = 1e-4,
   weightDecay = 1e-3,
   momentum = 1e-4
}

In [57]:
x, dl_dx = model:getParameters()

In [58]:
step = function(batch_size)
    local current_loss = 0
    local count = 0
    batch_size = batch_size or 200
    for t = 1, trainset.size, batch_size do
        local size = math.min(t + batch_size -1, trainset.size) - t
        local inputs = torch.Tensor(size, 20, 20)
        local targets = torch.Tensor(size)
        for i = 1,size do
            local input = trainset.data[i+t]
            local target = trainset.label[i+t]
            -- if target == 0 then target = 10 end
            inputs[i] = input
            targets[i] = target
        end
        
        local feval = function(x_new)
            -- reset data
            if x ~= x_new then x:copy(x_new) end
            dl_dx:zero()

            -- perform mini-batch gradient descent
            local loss = criterion:forward(model:forward(inputs), targets)
            model:backward(inputs, criterion:backward(model.output, targets))

            return loss, dl_dx
        end
        
        _, fs = optim.sgd(feval, x, sgd_params)
        -- fs is a table containing value of the loss function
        -- (just 1 value for the SGD optimization)
        count = count + 1
        current_loss = current_loss + fs[1]
    end

    -- normalize loss
    return current_loss / count
end

In [59]:
eval = function(dataset, batch_size)
    local count = 0
    batch_size = batch_size or 200
    
    for i = 1,dataset.size,batch_size do
        local size = math.min(i + batch_size - 1, dataset.size) - i
        local inputs = dataset.data[{{i,i+size-1}}]
        local targets = dataset.label[{{i,i+size-1}}]:long()
        local outputs = model:forward(inputs)
        local _, indices = torch.max(outputs, 2)
        local guessed_right = indices:eq(targets):sum()
        count = count + guessed_right
    end

    return count / dataset.size
end

In [62]:
max_iters = 150

In [63]:
do
    local Plot = require 'itorch.Plot'
    local last_accuracy = 0
    local decreasing = 0
    local loss={}
    local x={}
    local accuracy={}
    local threshold = 1 -- how many deacreasing epochs we allow
    for i = 1,max_iters do
        x[i]=i
        loss[i]=step()
        print(string.format('Epoch: %d Current loss: %4f', i, loss[i]))
        accuracy[i] =eval(validationset)
        print(string.format('Accuracy on the validation set: %4f', accuracy[i]))
               
        --[[if accuracy[i] < last_accuracy then
            if decreasing > threshold then break end
            decreasing = decreasing + 1
        else
            decreasing = 0
        end
        last_accuracy = accuracy[i]--]]
    end
    plot = Plot():circle(x, loss,'red','loss'):legend(true):title('Loss'):draw()
    plot1 = Plot():circle(x, accuracy,'green','accuracy'):legend(true):title('Accuacy'):draw()
    plot = Plot():line(x, loss,'red','loss'):legend(true):title('Loss'):draw()
    plot1 = Plot():line(x, accuracy,'green','accuracy'):legend(true):title('Accuacy'):draw()
end

Epoch: 1 Current loss: 1.041523	


Accuracy on the validation set: 0.731357	


Epoch: 2 Current loss: 0.983621	


Accuracy on the validation set: 0.736452	


Epoch: 3 Current loss: 0.939734	

...

Accuracy on the validation set: 0.887911	


Epoch: 149 Current loss: 0.355677	


Accuracy on the validation set: 0.891153	


Epoch: 150 Current loss: 0.352961	


Accuracy on the validation set: 0.892543	


In [19]:
eval(testset)

0.827	


In [20]:
torch.save('oldmodel.net',model)

In [21]:
plot:save('plot.html')
plot1:save('plot1.html')

In [20]:
eval(trainset)

0.83	


In [21]:
eval(validationset)

0.80824455766559	


In [28]:
a=image.load("r3.png",1,'byte')
--a=testset.data[10]

In [29]:
b=torch.Tensor(1,20,20)
b[1]=image.scale(a,20,20)
itorch.image(b[1])

In [30]:
pred=model:forward(b)
conf, indi=torch.sort(pred, true)
alpha=48
beta=65
gamma=indi[1][1]
if gamma < 11 then
    print(string.char(alpha+gamma-1))
else
    beta=beta+gamma-11
    print(string.char(beta))
end
--t=torch.Int(1):copy(1)
--indi[1]=indi[1]:type('int')
--print (CharTensor:(indi[1]))

R	
